# Generated Poetry in the Style of Edgar Allen Shakespeare

### Name: Kevin Liu

The purpose of this project is to generate a piece of poetry which falls under a similar style to that of Edgar Allen Poe and William Shakespeare. My initial expectations are that the poem output will be garbled at first, especially with a lower epoch count, but will initially become more comprehensible as epochs increase and we include dropout. We will be using a LSTM (Long Short Term Memory) RNN model, as it fits the purpose of the project. Using LSTM allows the model to remember import textual aspects, and forget unnecessary components. We are also using character-level generation as opposed to word-level, mainly due to lack of computational resources.

## 1. Data Preparation

We first import necessary libraries and Keras components.

In [96]:
import numpy as np
import pandas as pd
import random
import sys
import io
import re

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils

We load in the corpus containing Edgar Allen Poe's works and Shakespeare's sonnets (retrieved from gutenberg.org). We also will replace redundant characters to decrease the amount of unique characters we have to account for. We convert all letters to lowercase for the same reason.

In [206]:
path = "C:/Users/Ups/Desktop/Term Project Deep Learning/poe_shakespeare.txt" # path to corpus
with io.open(path, encoding='utf-8') as f: # read corpus into text string variable
    text = f.read().lower() # convert all text to lowercase so network does not have to account for capitalization

text = re.sub("æ|é|ê|ë|î|ï|û|\n|-|0|1|2|3|4|5|6|7|8|9|:|_|]|\'|/|", "", text)
text = text.replace("*", "")
text = text.replace("[", "")

In [207]:
len(text) # num of characters in text

235687

In [208]:
text

'the raven.  once upon a midnight dreary, while i pondered, weak and weary,  over many a quaint and curious volume of forgotten lore  while i nodded, nearly napping, suddenly there came a tapping,  as of some one gently rappingrapping at my chamber door.  "tis some visitor," i muttered, "tapping at my chamber door          only this and nothing more."  ah, distinctly i remember, it was in the bleak december,  and each separate dying ember wrought its ghost upon the floor.  eagerly i wished the morrow;vainly i had sought to borrow  from my books surcease of sorrowsorrow for the lost lenore  for the rare and radiant maiden whom the angels name lenore          nameless here for evermore.  and the silken sad uncertain rustling of each purple curtain  thrilled mefilled me with fantastic terrors never felt before;  so that now, to still the beating of my heart, i stood repeating  "tis some visitor entreating entrance at my chamber door  some late visitor entreating entrance at my chamber doo

In [209]:
# 53 unique characters in the corpus
characters = sorted(list(set(text)))
len(characters)

35

We utilize encoding to convert the text (characters) into integer values for the network to comprehend successfully (as a dictionary).

In [210]:
char_to_int = dict((c, i) for i, c in enumerate(characters)) # converts the text (chars) into indices
int_to_char = dict((i, c) for i, c in enumerate(characters)) # converts the indices into text (chars)

In [211]:
sentences = [] # store sentences
next_character = [] # next character

In [212]:
for i in range(0, len(text) - 40, 3): # goes through corpus and enumerates
    sentences.append(text[i:i+40])
    next_character.append(text[i+40])
    
x = np.zeros((len(sentences), 40, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

In [213]:
for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        x[i, j, char_to_int[char]] = 1
    y[i, char_to_int[next_character[i]]] = 1

## 2. Compile and Train Model

We start off by compiling a model with 1 LSTM layer, and 1 additional Dense layer with neuron count equal to the amount of unique characters in the text (52). We use a softmax activation, and loss function categorical crossentropy. The optimizer is RMSprop with learning rate of 0.01.

In [217]:
# create LSTM RNN model
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(40, len(characters)))) # Long Short Term Memory model (remembers important aspects and forgets unnecessary info)
model.add(keras.layers.Dense(len(characters),activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.01))

We fit the model using a batch size of 18, and only 1 epoch for time purposes.

In [218]:
model.fit(x,y,batch_size=18,epochs=1)

Epoch 1/1
78549/78549 [==============================] - 248s 3ms/step - loss: 2.0812


## 3. Test Model (Generate Poem)

In [219]:
def sample_generator(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    probes = np.random.multinomial(1, predictions, 1)
    return np.argmax(probes)

In [220]:
start_index = random.randint(0, len(text)-40-1)
for n in [0.2, 0.5, 1.0, 1.2]:
    generated = ""
    sentence = text[start_index: start_index+40]
    generated+=sentence
    for m in range(600):
        x_pred = np.zeros((1,40,len(characters)))
        for t, char in enumerate(sentence):
            x_pred[0,t,char_to_int[char]]=1
        predictions = model.predict(x_pred, verbose=0)[0]
        next_index = sample_generator(predictions, n)
        next_char = int_to_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char

The output poem below is barely comprehensible, but surprisingly contains some words that make sense. We could improve the model by either adding more dense layers, incorporating neuron dropout, adjusting batch size, or increasing the number of epochs. It would be ideal to have a much larger number of epochs, but training time would increase substantially. We could also change our optimizer to ADAM (Adaptive Moment Optimization) from RMSProp. We could also increase the number of hidden units but it will increase resource load and may cause over-fitting.

In [221]:
generated

' loathsome canker lives in sweetest bud. gufies emare me wilime so friest lobg mystos, why stiting mostit kithr out maght the will in spediparmessieg baces of thring tafn bire bregfaroulliaa parts away if lienetcept muunhtite swiles the enever more. no afrealfn the noun gidyersicg. i elvrly tiren milst be tandw  and kgewirty and dlobly lovicfy startled, chcaining senor  who come for toone i night oage noaghed mpyetd makead hee foa drombll liveihs moieed,  thot thou bet,           youre, and loid dim antip reainisanced laig  and a laopy great withrowf  cfor ome wo sums yet thonge the thathluth will thine, to heorknohel anafiuted wo t'

## 4. Compile Second Model and Generate New Poem

We will add more LSTM layers to the network, as well as increase the number of epochs to 8. We will also try using the adam optimizer rather than RMSProp.

In [134]:
# create LSTM RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(40, len(characters)), return_sequences=True)) # Long Short Term Memory model (remembers important aspects and forgets unnecessary info)
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(len(characters),activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [135]:
model.fit(x,y,batch_size=18,epochs=8) # try 8 epochs

Epoch 1/8
82321/82321 [==============================] - 950s 12ms/step - loss: 2.5517
Epoch 2/8
82321/82321 [==============================] - 930s 11ms/step - loss: 2.1473
Epoch 3/8
82321/82321 [==============================] - 987s 12ms/step - loss: 2.0000
Epoch 4/8
82321/82321 [==============================] - 1970s 24ms/step - loss: 1.9133
Epoch 5/8
82321/82321 [==============================] - 1977s 24ms/step - loss: 1.8503
Epoch 6/8
82321/82321 [==============================] - 1274s 15ms/step - loss: 1.7958
Epoch 7/8
82321/82321 [==============================] - 817s 10ms/step - loss: 1.7573
Epoch 8/8
82321/82321 [==============================] - 835s 10ms/step - loss: 1.7223


In [136]:
start_index = random.randint(0, len(text)-40-1)
for n in [0.2, 0.5, 1.0, 1.2]:
    generated = ""
    sentence = text[start_index: start_index+40]
    generated+=sentence
    for m in range(600):
        x_pred = np.zeros((1,40,len(characters)))
        for t, char in enumerate(sentence):
            x_pred[0,t,char_to_int[char]]=1
        predictions = model.predict(x_pred, verbose=0)[0]
        next_index = sample_generator(predictions, n)
        next_char = int_to_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char

In [137]:
generated

' yet, love to tell me so;--\n as testy sight dly thought pleate ceul dadit the.--\n  "her hank vairel ghon\'s.\n\n yye:\n\n  (s\n\nwutiliiniaf\'s ghastanils trustiinang reed praash dream!)\n  with movine-need sward:\n of fromest i sell; or deel \'is, gnave:\n or that full bs the ad8 haves then you mrowns;\n come _neet virle ligh beep and in satsom;\n  id encoth mirr. their baid burtite nom\'ts amrof pu\'he in expine all theel alantiyne\n\n  rumanted my swiers, rofes\ntallly statescunly!-daany jave would deepsest all a bnocelosge:\n ceet, name lagod and dementlish clless,\n which is the pregipuld youn", wost oriam;\n o hown hy, to give;;old from, preevys\n  '

## 5. Compile Third Model and Generate New Poem

In [235]:
# create LSTM RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(40, len(characters)))) # Long Short Term Memory model (remembers important aspects and forgets unnecessary info)
model.add(Dense(len(characters),activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.001))

In [236]:
model.fit(x,y,batch_size=128,epochs=20)

Epoch 1/20
78549/78549 [==============================] - 189s 2ms/step - loss: 2.6153
Epoch 2/20
78549/78549 [==============================] - 198s 3ms/step - loss: 2.2654
Epoch 3/20
78549/78549 [==============================] - 202s 3ms/step - loss: 2.1371
Epoch 4/20
78549/78549 [==============================] - 206s 3ms/step - loss: 2.0536
Epoch 5/20
78549/78549 [==============================] - 195s 2ms/step - loss: 1.9912
Epoch 6/20
78549/78549 [==============================] - 211s 3ms/step - loss: 1.9397
Epoch 7/20
78549/78549 [==============================] - 200s 3ms/step - loss: 1.8952
Epoch 8/20
78549/78549 [==============================] - 177s 2ms/step - loss: 1.8565
Epoch 9/20
78549/78549 [==============================] - 178s 2ms/step - loss: 1.8226
Epoch 10/20
78549/78549 [==============================] - 201s 3ms/step - loss: 1.7909
Epoch 11/20
78549/78549 [==============================] - 179s 2ms/step - loss: 1.7618
Epoch 12/20
78549/78549 [================

In [241]:
start_index = random.randint(0, len(text)-40-1)
for n in [0.2, 0.5, 1.0, 1.2]:
    generated = ""
    sentence = text[start_index: start_index+40]
    generated+=sentence
    for m in range(600):
        x_pred = np.zeros((1,40,len(characters)))
        for t, char in enumerate(sentence):
            x_pred[0,t,char_to_int[char]]=1
        predictions = model.predict(x_pred, verbose=0)[0]
        next_index = sample_generator(predictions, n)
        next_char = int_to_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char

The generated poem below has more understandable words, such as: port, right, was, but, in, who, thought, when, sour - as well as some common poetry-related words like thy. There are also some words that are missing one or two particular letters to become actual words as well.

In [242]:
generated

'  my own had past, did not the beam  of beart out worst syelick otherwhe hile ut, as wheret!   paseupte stoks the river, thought, port becard, at whoch right lasse was but tonotiencerralidig in apbarppiatet no  who a womy swreep to presenese, brooks! it thou hose or fleste, wherefels thought museld when eyd, thy sour verlys urfey and, onem, to utterbet fere, enged, bralls reeve cupby lins!,hith a truetws of where, has true that is e fere. ih haw  nouths becting thy crime, a leatalail. bish that with the inortinionerghitpoms, whec oster not men ever enojimy that his londure o her sairs, the simmin lumbe, astersiin the paysperfupon at'

We will try decreasing the learning rate even further down to 0.0001.

In [243]:
# create LSTM RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(40, len(characters)))) # Long Short Term Memory model (remembers important aspects and forgets unnecessary info)
model.add(Dense(len(characters),activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.0001))

In [244]:
model.fit(x,y,batch_size=128,epochs=100) # try 100 epochs

Epoch 1/100
78549/78549 [==============================] - 201s 3ms/step - loss: 2.9210
Epoch 2/100
78549/78549 [==============================] - 201s 3ms/step - loss: 2.8222
Epoch 3/100
78549/78549 [==============================] - 247s 3ms/step - loss: 2.7723
Epoch 4/100
78549/78549 [==============================] - 214s 3ms/step - loss: 2.7074
Epoch 5/100
78549/78549 [==============================] - 196s 2ms/step - loss: 2.6314
Epoch 6/100
78549/78549 [==============================] - 199s 3ms/step - loss: 2.5560
Epoch 7/100
78549/78549 [==============================] - 196s 2ms/step - loss: 2.4914
Epoch 8/100
78549/78549 [==============================] - 184s 2ms/step - loss: 2.4402
Epoch 9/100
78549/78549 [==============================] - 187s 2ms/step - loss: 2.4011
Epoch 10/100
78549/78549 [==============================] - 264s 3ms/step - loss: 2.3698
Epoch 11/100
78549/78549 [==============================] - 433s 6ms/step - loss: 2.3443
Epoch 12/100
78549/78549 [====

Although the loss is steadily decreasing, having the learning rate at such an extremely low value will force us to run an enormous number of epochs before we reach a suitable loss amount.

## 6. Final Model - 125 Epochs

We will now try 125 epochs with the previous learning rate of 0.001 as opposed to 0.0001.

In [250]:
# create LSTM RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(40, len(characters)))) # Long Short Term Memory model (remembers important aspects and forgets unnecessary info)
model.add(Dense(len(characters),activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.001))

In [251]:
model.fit(x,y,batch_size=128, epochs=125)

Epoch 1/125
78549/78549 [==============================] - 187s 2ms/step - loss: 2.6089
Epoch 2/125
78549/78549 [==============================] - 173s 2ms/step - loss: 2.2709
Epoch 3/125
78549/78549 [==============================] - 181s 2ms/step - loss: 2.1447
Epoch 4/125
78549/78549 [==============================] - 183s 2ms/step - loss: 2.0616
Epoch 5/125
78549/78549 [==============================] - 192s 2ms/step - loss: 1.9986
Epoch 6/125
78549/78549 [==============================] - 178s 2ms/step - loss: 1.9471
Epoch 7/125
78549/78549 [==============================] - 149s 2ms/step - loss: 1.9020
Epoch 8/125
78549/78549 [==============================] - 149s 2ms/step - loss: 1.8623
Epoch 9/125
78549/78549 [==============================] - 149s 2ms/step - loss: 1.8271
Epoch 10/125
78549/78549 [==============================] - 149s 2ms/step - loss: 1.7958
Epoch 11/125
78549/78549 [==============================] - 148s 2ms/step - loss: 1.7657
Epoch 12/125
78549/78549 [====

In [301]:
start_index = random.randint(0, len(text)-40-1)
for n in [0.2, 0.5, 1.0, 1.2]:
    generated = ""
    sentence = text[start_index: start_index+40]
    generated+=sentence
    for m in range(600):
        x_pred = np.zeros((1,40,len(characters)))
        for t, char in enumerate(sentence):
            x_pred[0,t,char_to_int[char]]=1
        predictions = model.predict(x_pred, verbose=0)[0]
        next_index = sample_generator(predictions, n)
        next_char = int_to_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char

The poem output is now much clearer, and even has proper poetic structure. Although there are still some garbled words, no words are being repeated, and punctuation is placed in correct areas. 

In [302]:
generated

'e, thou shalt not boast that i do change slaxded and host chardlys, look, and the rymas befrieved my sain;   nor the rostly shill wricked the moons arow, and me, thou is a pidces it what the winding of the rine. i in for shide have upon the wild, aw out to the disporblin place with xave my though the airs, ho contrinted for that . imertraines in pal uemphine as exits  dirto so rosoks of to. .  mes! he starll it wingus in starp!  for i have they vertail oignth the myre that imall thine apeisty soulmed at acs, with to crise is dumber, the nigituess for theneusting and the eirthings, charvs your gend, like but in the treblk will grows '

In [278]:
model.summary()

Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_77 (LSTM)               (None, 128)               83968     
_________________________________________________________________
dense_42 (Dense)             (None, 35)                4515      
Total params: 88,483
Trainable params: 88,483
Non-trainable params: 0
_________________________________________________________________


In [303]:
# create LSTM RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(40, len(characters)))) # Long Short Term Memory model (remembers important aspects and forgets unnecessary info)
model.add(Dense(len(characters),activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.001))

In [304]:
model.fit(x,y,batch_size=128, epochs=500)

Epoch 1/500
78549/78549 [==============================] - 143s 2ms/step - loss: 2.6009
Epoch 2/500
78549/78549 [==============================] - 142s 2ms/step - loss: 2.2618
Epoch 3/500
78549/78549 [==============================] - 145s 2ms/step - loss: 2.1319
Epoch 4/500
78549/78549 [==============================] - 150s 2ms/step - loss: 2.0482
Epoch 5/500
78549/78549 [==============================] - 156s 2ms/step - loss: 1.9853
Epoch 6/500
78549/78549 [==============================] - 163s 2ms/step - loss: 1.9341
Epoch 7/500
78549/78549 [==============================] - 177s 2ms/step - loss: 1.8901
Epoch 8/500
78549/78549 [==============================] - 177s 2ms/step - loss: 1.8516
Epoch 9/500
78549/78549 [==============================] - 156s 2ms/step - loss: 1.8175
Epoch 10/500
78549/78549 [==============================] - 158s 2ms/step - loss: 1.7854
Epoch 11/500
78549/78549 [==============================] - 139s 2ms/step - loss: 1.7556
Epoch 12/500
78549/78549 [====

78549/78549 [==============================] - 185s 2ms/step - loss: 0.6920
Epoch 185/500
78549/78549 [==============================] - 157s 2ms/step - loss: 0.6897
Epoch 186/500
78549/78549 [==============================] - 150s 2ms/step - loss: 0.6907
Epoch 187/500
78549/78549 [==============================] - 150s 2ms/step - loss: 0.6927
Epoch 188/500
78549/78549 [==============================] - 160s 2ms/step - loss: 0.6873
Epoch 189/500
78549/78549 [==============================] - 149s 2ms/step - loss: 0.6889
Epoch 190/500
78549/78549 [==============================] - 161s 2ms/step - loss: 0.6888
Epoch 191/500
78549/78549 [==============================] - 151s 2ms/step - loss: 0.6848
Epoch 192/500
78549/78549 [==============================] - 151s 2ms/step - loss: 0.6863
Epoch 193/500
78549/78549 [==============================] - 161s 2ms/step - loss: 0.6818
Epoch 194/500
78549/78549 [==============================] - 159s 2ms/step - loss: 0.6830
Epoch 195/500
78549/7854

78549/78549 [==============================] - 142s 2ms/step - loss: 0.6184
Epoch 367/500
78549/78549 [==============================] - 142s 2ms/step - loss: 0.6134
Epoch 368/500
78549/78549 [==============================] - 142s 2ms/step - loss: 0.6132
Epoch 369/500
78549/78549 [==============================] - 143s 2ms/step - loss: 0.6148
Epoch 370/500
78549/78549 [==============================] - 141s 2ms/step - loss: 0.6155
Epoch 371/500
78549/78549 [==============================] - 142s 2ms/step - loss: 0.6183
Epoch 372/500
78549/78549 [==============================] - 142s 2ms/step - loss: 0.6164
Epoch 373/500
78549/78549 [==============================] - 142s 2ms/step - loss: 0.6175
Epoch 374/500
78549/78549 [==============================] - 142s 2ms/step - loss: 0.6169
Epoch 375/500
78549/78549 [==============================] - 142s 2ms/step - loss: 0.6154
Epoch 376/500
78549/78549 [==============================] - 143s 2ms/step - loss: 0.6180
Epoch 377/500
78549/7854

In [315]:
start_index = random.randint(0, len(text)-40-1)
for n in [0.2, 0.5, 1.0, 1.2]:
    generated = ""
    sentence = text[start_index: start_index+40]
    generated+=sentence
    for m in range(600):
        x_pred = np.zeros((1,40,len(characters)))
        for t, char in enumerate(sentence):
            x_pred[0,t,char_to_int[char]]=1
        predictions = model.predict(x_pred, verbose=0)[0]
        next_index = sample_generator(predictions, n)
        next_char = int_to_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char

In [316]:
generated

'ule  with a despotic sway all giant minds of the rulling, wing wind would galling crivin.  for hir of the wastrring drow  sight,  my spirit not the strem of your gract      poriss bester,  where thy weads respok, far thou huspond, hand did liver love therefore har, he deed to lowi ahy dimptance, from the shadow rime;  , shelly gleath mady dost more that beautys, by a dream of hes right breathouthis by and away and hea, louddon thou prust pute shadow, is ompoarm, which i coscest; fars, from chearestencjuchm dury  waince, to without most foround. xyi agefles did epen forming; that have hope ther part have.                             '

We can now try a deep stacked LSTM model, with multiple hidden layers, which will end up being a more difficult optimization problem to solve, but has the potential to reach a much lower minimum loss compared to the single LSTM model.

In [318]:
# create LSTM RNN model
model_deep = Sequential()
model_deep.add(LSTM(128, input_shape=(40, len(characters)), return_sequences=True)) # Long Short Term Memory model (remembers important aspects and forgets unnecessary info)
model_deep.add(Dropout(0.1))
model_deep.add(LSTM(128, return_sequences=True))
model_deep.add(Dropout(0.1))
model_deep.add(LSTM(128, return_sequences=True))
model_deep.add(Dropout(0.1))
model_deep.add(LSTM(128))
model_deep.add(Dense(len(characters),activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.001))

## 6. Conclusion and Future Improvements

There are some components we could further tweak to improve generation accuracy, some of which would increase resource load significantly.
Some possible changes could be:
- Increase number of epochs even more (500+), and save the point where loss is at its minimum
- Tweak learning rate to be between 0.01 and 0.001
- Change batch size
- Increase the number of LSTM layers
- Tweak dropout percentages (between 0.1 and 0.25)
- Clean up sentence structure and words in corpus